### Original data: https://dadosabertos.mec.gov.br/prouni

### Importing Packages

In [1]:
import os
import pandas as pd
import numpy as np
import time
from dask import dataframe as dd
from colabcode import ColabCode
import glob

### Setting working directory

In [4]:
os.chdir(r'/Users/lucaspb/git-repositories/portfolio-projects/python/basedosdados/bases/br_mec_sisu/espera')

### Knowing the Dataset

In [6]:
filename = 'input/ListagemListaEspera_2020-1.csv'

In [13]:
start = time.time()
df = dd.read_csv(filename, encoding='latin-1', header=None, sep='\t')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.01772475242614746 sec


In [15]:
df.head(2)

,0
0,"NU_ANO|""NU_EDICAO""|""CO_ETAPA""|""DS_ETAPA""|""CO_I..."
1,"2020|1|7|""LISTA DE ESPERA""|548|""UNIVERSIDADE F..."


In [17]:
start = time.time()
df = dd.read_csv(filename, encoding='latin-1', header=0, sep='|', dtype='string')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.02245783805847168 sec


In [28]:
len(df)

882359

In [27]:
df.head()

,NU_ANO,NU_EDICAO,CO_ETAPA,DS_ETAPA,CO_IES,NO_IES,SIGLA_IES,SG_UF_IES,CO_CAMPUS,NO_CAMPUS,...,NU_NOTA_R,NOTA_L_COM_PESO,NOTA_CH_COM_PESO,NOTA_CN_COM_PESO,NOTA_M_COM_PESO,NOTA_R_COM_PESO,NU_NOTA_CANDIDATO,NU_NOTACORTE,NU_CLASSIFICACAO,ST_APROVADO
0,2020,1,7,LISTA DE ESPERA,548,UNIVERSIDADE FEDERAL DO MARANHÃO,UFMA,MA,1006314,Cidade Universitária,...,480,"434,6","512,2","386,8","552,1",480,"473,14","579,74",132,S
1,2020,1,7,LISTA DE ESPERA,548,UNIVERSIDADE FEDERAL DO MARANHÃO,UFMA,MA,1006314,Cidade Universitária,...,620,"527,6","541,4","498,2",454,620,"528,24","579,74",56,S
2,2020,1,7,LISTA DE ESPERA,548,UNIVERSIDADE FEDERAL DO MARANHÃO,UFMA,MA,1006314,Cidade Universitária,...,600,"489,9","419,1","471,3","544,6",600,"504,98","579,74",85,S
3,2020,1,7,LISTA DE ESPERA,548,UNIVERSIDADE FEDERAL DO MARANHÃO,UFMA,MA,1006314,Cidade Universitária,...,580,"549,6","425,2","431,5","465,4",580,"588,41","694,22",137,S
4,2020,1,7,LISTA DE ESPERA,548,UNIVERSIDADE FEDERAL DO MARANHÃO,UFMA,MA,1006314,Cidade Universitária,...,540,"559,1",515,497,674,540,"557,02","579,74",24,S


In [41]:
print(df.columns)

Index(['NU_ANO', 'NU_EDICAO', 'CO_ETAPA', 'DS_ETAPA', 'CO_IES', 'NO_IES',
       'SIGLA_IES', 'SG_UF_IES', 'CO_CAMPUS', 'NO_CAMPUS', 'SG_UF_CAMPUS',
       'NO_MUNUCIPIO_CAMPUS', 'CO_IES_CURSO', 'NO_CURSO', 'DS_GRAU',
       'DS_TURNO', 'TP_MOD_CONCORRENCIA', 'DS_MOD_CONCORRENCIA',
       'NU_PERCENTUAL_BONUS', 'NU_PESO_L', 'NU_PESO_CH', 'NU_PESO_CN',
       'NU_PESO_M', 'NU_PESO_R', 'NOTA_MINIMA_L', 'NOTA_MINIMA_CH',
       'NOTA_MINIMA_CN', 'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA',
       'NU_CPF', 'CO_INSCRICAO_ENEM', 'NO_INSCRITO', 'TP_SEXO',
       'DT_NASCIMENTO', 'SG_UF_CANDIDATO', 'MUNICIPIO_CANDIDATO', 'ST_OPCAO',
       'NU_NOTA_L', 'NU_NOTA_CH', 'NU_NOTA_CN', 'NU_NOTA_M', 'NU_NOTA_R',
       'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO', 'NOTA_CN_COM_PESO',
       'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO', 'NU_NOTA_CANDIDATO',
       'NU_NOTACORTE', 'NU_CLASSIFICACAO', 'ST_APROVADO'],
      dtype='object')


In [36]:
print(set(df['NU_ANO']))

{'2020'}


In [37]:
print(set(df['NU_EDICAO']))

{'1'}


In [39]:
print(set(df['CO_ETAPA']))

{'7'}


In [40]:
print(set(df['ST_APROVADO']))

{'S'}


In [42]:
df['TP_MOD_CONCORRENCIA'].head()

0    L
1    L
2    L
3    B
4    L
Name: TP_MOD_CONCORRENCIA, dtype: string

In [43]:
print(set(df['TP_MOD_CONCORRENCIA']))

{'V', 'B', 'A', 'L'}


In [4]:
def getstuff(filename, criterion):
    with open(filename, "rb") as csvfile:
        datareader = csv.reader(csvfile)
        yield next(datareader)  # yield the header row
        # first row, plus any subsequent rows that match, then stop
        # reading altogether
        # Python 2: use `for row in takewhile(...): yield row` instead
        # instead of `yield from takewhile(...)`.
        yield from takewhile(
            lambda r: r[3] == criterion,
            dropwhile(lambda r: r[3] != criterion, datareader))
        return

In [5]:
def getdata(filename, criteria):
    for criterion in criteria:
        for row in getstuff(filename, criterion):
            yield row

In [3]:
for row in getdata('input/ListagemListaEspera_2020-1.csv', sequence_of_criteria):

IndentationError: expected an indented block (<ipython-input-3-2eab46da0db1>, line 1)

### Setting parameters for renaming and reordering of columns

In [3]:
rename = {'ANO_CONCESSAO_BOLSA':'ano', 
          'ï»¿ANO_CONCESSAO_BOLSA':'ano',
          '´╗┐ANO_CONCESSAO_BOLSA':'ano',
          'UF_BENEFICIARIO':'sigla_uf',
          'SIGLA_UF_BENEFICIARIO_BOLSA':'sigla_uf',
          'MUNICIPIO_BENEFICIARIO':'id_municipio',
          'MUNICIPIO_BENEFICIARIO_BOLSA':'id_municipio',
          'CPF_BENEFICIARIO':'cpf',
          'CPF_BENEFICIARIO_BOLSA':'cpf',
          'SEXO_BENEFICIARIO':'sexo',
          'SEXO_BENEFICIARIO_BOLSA':'sexo',
          'RACA_BENEFICIARIO':'raca_cor',
          'RACA_BENEFICIARIO_BOLSA':'raca_cor',
          'DATA_NASCIMENTO':'data_nascimento',
          'DT_NASCIMENTO_BENEFICIARIO':'data_nascimento',
          'BENEFICIARIO_DEFICIENTE_FISICO':'beneficiario_deficiente',
          'CODIGO_EMEC_IES_BOLSA':'id_ies',
          'CAMPUS':'campus',
          'MUNICIPIO':'id_municipio_ies',
          'NOME_CURSO_BOLSA':'curso',
          'NOME_TURNO_CURSO_BOLSA':'turno_curso',
          'TIPO_BOLSA':'tipo_bolsa',
          'MODALIDADE_ENSINO_BOLSA':'modalidade_ensino' }

ordem2020 = ['ano', 'sigla_uf', 'id_municipio', 'cpf', 'sexo', 'raca_cor', 'data_nascimento',
         'beneficiario_deficiente', 'id_ies', 'campus', 'id_municipio_ies', 'curso', 'turno_curso', 'tipo_bolsa', 'modalidade_ensino']

### Settings parameters for partitioning the output in the end

In [4]:
anos = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

ufs = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 
       'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

### Creating folders for output partitioning

In [37]:
for ano in anos:
  for uf in ufs:
    directory = 'output/ano={}/sigla_uf={}'.format(ano, uf)
    if not os.path.exists(directory):
      os.makedirs(directory)

#### Getting CSV files list from my folder
##### 2017,2018 and 2019 are in another encoding format

In [5]:
path1 = 'input/1'
path2 = 'input/2'
csv_files1 = glob.glob(path1 + "/*.csv")
csv_files2 = glob.glob(path2 + "/*.csv")

In [6]:
print(csv_files1)

['input/1/pda-prouni-2009.csv', 'input/1/pda-prouni-2008.csv', 'input/1/pda-prouni-2016.csv', 'input/1/pda-prouni-2014.csv', 'input/1/pda-prouni-2015.csv', 'input/1/pda-prouni-2005.csv', 'input/1/pda-prouni-2011.csv', 'input/1/pda-prouni-2010.csv', 'input/1/pda-prouni-2012.csv', 'input/1/pda-prouni-2006.csv', 'input/1/pda-prouni-2007.csv', 'input/1/pda-prouni-2013.csv']


In [40]:
print(csv_files2)

['input/2/pda-prouni-2020.csv', 'input/2/pda-prouni-2018.csv', 'input/2/pda-prouni-2019.csv', 'input/2/pda-prouni-2017.csv']


### Reading into a list of dataframes

In [ ]:
dfs = [pd.read_csv(file, sep=";", encoding='latin-1', header=0) for file in csv_files1]
dfs2 = [pd.read_csv(file, sep=";", encoding='utf-8', header=0) for file in csv_files2]

In [169]:
dfs.extend(dfs2)

In [145]:
dfs[14].head(1)

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO_BOLSA,SEXO_BENEFICIARIO_BOLSA,RACA_BENEFICIARIO_BOLSA,DT_NASCIMENTO_BENEFICIARIO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO_BOLSA,SIGLA_UF_BENEFICIARIO_BOLSA,MUNICIPIO_BENEFICIARIO_BOLSA
0,2019.0,10.0,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO PARANÁ,BOLSA INTEGRAL,EAD,Administração,Curso a distância,***03183970**,M,Branca,19/08/1993,N,Sul,PR,LONDRINA


Checking all years because column was being deleted

In [17]:
list_ano = []

for i in range(len(dfs)):
    list_ano.append(int(dfs[i].iloc[0, 0]))

list_ano.sort()
print(list_ano)

[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


This could be made into one bit dataframe, but I've decided not to go on this route
prounidata = pd.concat(dfs,ignore_index=True)

## Cleaning data

In [170]:
for df in dfs:
    df.rename(columns=rename, inplace=True)
    df.drop(['NOME_IES_BOLSA', 'REGIAO_BENEFICIARIO_BOLSA', 'REGIAO_BENEFICIARIO'], errors='ignore', axis=1, inplace=True)
    df['sexo'] = df['sexo'].str.upper().replace({'FEMININO':'F', 'MASCULINO':'M'})
    df['sexo'] = df['sexo'].replace({'F':'1', 'M':'2'})
    df['raca_cor'] = df['raca_cor'].str.upper().replace({'AMARELA':'1', 'BRANCA':'2', 'INDÍGENA':'3', 'PARDA':'4', 'PRETA':'5', 'NÃO INFORMADA':'6'})
    df['beneficiario_deficiente'] = df['beneficiario_deficiente'].str.upper().replace({'SIM':'S', 'NÃO':'N'})
    df['beneficiario_deficiente'] = df['beneficiario_deficiente'].str.upper().replace({'S':'1', 'N':'0'})
    df['turno_curso'] = df['turno_curso'].str.upper().replace({'MATUTINO':'1', 'VESPERTINO':'2', 'NOTURNO':'3', 'INTEGRAL':'4', 'CURSO A DISTÂNCIA':'5'})
    df['tipo_bolsa'] = df['tipo_bolsa'].str.upper().replace({'BOLSA INTEGRAL':'1', 'BOLSA PARCIAL 50%':'2', 'BOLSA COMPLEMENTAR 25%':'3'})
    df['tipo_bolsa'] = df['tipo_bolsa'].str.upper().replace({'INTEGRAL':'1', 'PARCIAL':'2'})
    df['modalidade_ensino'] = df['modalidade_ensino'].str.upper().replace({'PRESENCIAL':'1', 'EDUCAÇÃO A DISTÂNCIA':'2', 'EAD':'2'})
    df['data_nascimento'] = pd.to_datetime(df['data_nascimento'], errors='coerce')
    df['data_nascimento'] = df['data_nascimento'].dt.strftime('%Y-%m-%d')

In [171]:
for df in dfs:
    df['turno_curso'] = df['turno_curso'].str.upper().replace({'CURSO A DISTÃ¢NCIA':'5'})
    df['turno_curso'] = df['turno_curso'].str.upper().replace({'CURSO A DISTÃ¢NCIA':'5'})
    df['turno_curso'] = df['turno_curso'].str.upper().replace({'CURSO A DISTNCIA':'5'})
    df['raca_cor'] = df['raca_cor'].str.upper().replace({'IND¡GENA':'3', 'NÆO INFORMADA':'6'})
    df['raca_cor'] = df['raca_cor'].str.upper().replace({'INDÃ­GENA':'3', 'NÃ£O INFORMADA':'6'})

### Setting up to replace counties names for its unique id

#### Importing dataframe with counties and unique ids

In [172]:
id_mcp = pd.read_csv(r'/Users/lucaspb/git-repositories/portfolio-projects/python/basedosdados/assets/dir_municipio_uf.csv', sep=',', dtype='string', header=0)

#### Normalizing names

##### Upper Case

In [173]:
id_mcp = id_mcp.applymap(lambda s: s.upper() if type(s) == str else s)

In [174]:
for df in dfs:
    ano = int(df.iloc[0, 0])
    if ano != 2020:
        df['id_municipio'] = df['id_municipio'].apply(lambda s: s.upper() if type(s) == str else s)
    if ano == 2020: 
        df['id_municipio'] = df['id_municipio'].apply(lambda s: s.upper() if type(s) == str else s)
        df['id_municipio_ies'] = df['id_municipio_ies'].apply(lambda s: s.upper() if type(s) == str else s)

In [36]:
dfs[12].head(3)

,ano,id_ies,id_municipio_ies,campus,tipo_bolsa,modalidade_ensino,curso,turno_curso,cpf,sexo,raca_cor,data_nascimento,beneficiario_deficiente,sigla_uf,id_municipio
0,2020,322,IPATINGA,IPATINGA,1,2,PEDAGOGIA,5,991.XXX.XXX-91,1,4,1973-11-08,0,MG,GOVERNADOR VALADARES
1,2020,163,FORTALEZA,EAD VIA CORPVS - CE,1,2,MARKETING,5,067.XXX.XXX-01,2,4,1987-05-13,0,CE,FORTALEZA
2,2020,17670,QUIXERAMOBIM,FACULDADE DE QUIXERAMOBIM - UNIQ,1,1,FARMÁCIA,3,623.XXX.XXX-27,2,4,2001-07-23,0,CE,MOMBACA


##### Removing accentuation

In [175]:
cols = id_mcp.select_dtypes(include=[np.object]).columns
id_mcp[cols] = id_mcp[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
id_mcp['nome'] = id_mcp['nome'].apply(lambda x: x.replace("‘", ' ').replace("’", ' ').replace("'", ' ').replace('"', ' ').replace('`', ' ').replace('-', ' '))
id_mcp['nome'] = id_mcp['nome'].apply(lambda x: x.replace("SANT ANA DO LIVRAMENTO", 'SANTANA DO LIVRAMENTO'))

<ipython-input-175-b58b3d4df908>:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = id_mcp.select_dtypes(include=[np.object]).columns


In [176]:
for df in dfs:
    ano = int(df.iloc[0, 0])
    if ano != 2020:
        cols = df.select_dtypes(include=[np.object]).columns
        df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
        df['id_municipio'] = df['id_municipio'].apply(lambda x: x.replace("‘", ' ').replace("’", ' ').replace("'", ' ').replace('"', ' ').replace('`', ' ').replace('-', ' ') if type(x) == str else x)
    if ano == 2020: 
        cols = df.select_dtypes(include=[np.object]).columns
        df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
        df['id_municipio'] = df['id_municipio'].apply(lambda x: x.replace("‘", ' ').replace("’", ' ').replace("'", ' ').replace('"', ' ').replace('`', ' ').replace('-', ' ') if type(x) == str else x)
        df['id_municipio_ies'] = df['id_municipio_ies'].apply(lambda x: x.replace("‘", ' ').replace("’", ' ').replace("'", ' ').replace('"', ' ').replace('`', ' ').replace('-', ' ') if type(x) == str else x)

<ipython-input-176-fbcd1dfd0791>:4: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = df.select_dtypes(include=[np.object]).columns
<ipython-input-176-fbcd1dfd0791>:8: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = df.select_dtypes(include=[np.object]).columns


##### Fixing specific problems

In [177]:
for df in dfs:
    ano = int(df.iloc[0, 0])
    if ano != 2020:
        df['id_municipio'] = df['id_municipio'].apply(lambda x: x.replace("EMBU", 'EMBU DAS ARTES') if type(x) == str else x)
    if ano == 2020: 
        df['id_municipio'] = df['id_municipio'].apply(lambda x: x.replace("EMBU", 'EMBU DAS ARTES') if type(x) == str else x)
        df['id_municipio_ies'] = df['id_municipio_ies'].apply(lambda x: x.replace("EMBU", 'EMBU DAS ARTES') if type(x) == str else x)

##### Replacing counties names for its unique id

In [178]:
for i in range(len(dfs)):
    dfs[i] = pd.merge(id_mcp, dfs[i],
                    how='right', left_on=['sigla_uf', 'nome'], right_on=['sigla_uf', 'id_municipio'])

##### Reordering columns

In [179]:
for i in range(len(dfs)):
    ano = int(dfs[i].iloc[0, 3])
    if ano != 2020:
        dfs[i].rename(columns={'id_municipio_x':'id_municipio'}, inplace=True)
        dfs[i] = pd.concat([dfs[i],pd.DataFrame(columns=['campus', 'id_municipio_ies'])])
    if ano == 2020:
        dfs[i].rename(columns={'id_municipio_x':'id_municipio'}, inplace=True)

In [180]:
for i in range(len(dfs)):
    dfs[i] = dfs[i][ordem2020]

In [127]:
dfs[12].head(5)

,ano,sigla_uf,id_municipio,cpf,sexo,raca_cor,data_nascimento,beneficiario_deficiente,id_ies,campus,id_municipio_ies,curso,turno_curso,tipo_bolsa,modalidade_ensino
0,2020,MG,3127701,991.XXX.XXX-91,1,4,1973-11-08,0,322,IPATINGA,IPATINGA,PEDAGOGIA,5,1,2
1,2020,CE,2304400,067.XXX.XXX-01,2,4,1987-05-13,0,163,EAD VIA CORPVS - CE,FORTALEZA,MARKETING,5,1,2
2,2020,CE,2308500,623.XXX.XXX-27,2,4,2001-07-23,0,17670,FACULDADE DE QUIXERAMOBIM - UNIQ,QUIXERAMOBIM,FARMACIA,3,1,1
3,2020,BA,2913101,089.XXX.XXX-40,1,2,2003-04-04,0,203,PAULISTA,SAO PAULO,DIREITO,1,2,1
4,2020,SP,3550308,173.XXX.XXX-09,1,2,1977-07-12,0,203,PAULISTA,SAO PAULO,DIREITO,1,1,1


In [128]:
list_ano = []

for i in range(len(dfs)):
    list_ano.append(int(dfs[i].iloc[0, 0]))

list_ano.sort()
print(list_ano)

[2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


Now I would merge the id_municipio_ies with the table id_mcp to replace each name of municipio_ies for its id, but in order to do that, I need the sigla_uf which was provided by MEC just for the person and not the institution. I've tried to use id_ies to to that, but we have the same institution (with the same ID) running on different cities. 

Since its not possible to merge, I'll rename id_municipio_ies to nome_municipio_ies

In [181]:
for i in range(len(dfs)):
    dfs[i].rename(columns={'id_municipio_ies':'nome_municipio_ies'}, inplace=True)

Applying UPPER to 'curso', 'campus' and normalizing CPF. Did this down the line because it was a f
feedback from team. 

In [182]:
for df in dfs:
        df['curso'] = df['curso'].apply(lambda s: s.upper() if type(s) == str else s)
        df['campus'] = df['campus'].apply(lambda s: s.upper() if type(s) == str else s)
        df['cpf'] = df['cpf'].apply(lambda s: s.replace('X', '*') if type(s) == str else s)

#### Checking for duplicates and Droping it

In [183]:
for i in range(len(dfs)):
    print(len(dfs[i])-len(dfs[i].drop_duplicates()))
    # same as print(len(dfs[i][dfs[i].duplicated()]))

120
62
307
213
285
40
119
124
108
33
41
127
109
3155
17997
3198


Wow, one year has 17k duplicates!

In [184]:
dfs[14].head(3)

,ano,sigla_uf,id_municipio,cpf,sexo,raca_cor,data_nascimento,beneficiario_deficiente,id_ies,campus,nome_municipio_ies,curso,turno_curso,tipo_bolsa,modalidade_ensino
0,2019.0,PR,4113700,***03183970**,2,2,1993-08-19,0,10.0,NaN,NaN,ADMINISTRACAO,5,1,2
1,2019.0,PR,4125506,***92848904**,1,2,1999-03-11,0,10.0,NaN,NaN,ADMINISTRACAO,5,1,2
2,2019.0,PR,4120804,***12457929**,2,4,1999-02-04,0,10.0,NaN,NaN,ANALISE E DESENVOLVIMENTO DE SISTEMAS,5,1,2


In [115]:
list_duplicated = []

for i in range(len(dfs)):
    list_duplicated.append(dfs[i][dfs[i].duplicated(keep=False)])

In [ ]:
list_duplicated[11].sort_values(by=['cpf'], ascending=False)

In [51]:
for i in range(len(dfs)):
    dfs[i].drop_duplicates(inplace=True)

In [186]:
for i in range(len(dfs)):
    print(len(dfs[i])-len(dfs[i].drop_duplicates()))

612
294
1032
947
978
44
549
456
486
193
241
634
308
42181
50982
46948


ANALISE DE DUPLICATES

In [146]:
list_duplicated_preprocessing = []

for i in range(len(dfs)):
    list_duplicated_preprocessing.append(dfs[i][dfs[i].duplicated(keep=False)])

In [156]:
list_duplicated_preprocessing[12].sort_values(by=['CPF_BENEFICIARIO'], ascending=False)

,´╗┐ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,MUNICIPIO,CAMPUS,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO,SEXO_BENEFICIARIO,RACA_BENEFICIARIO,DATA_NASCIMENTO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO,UF_BENEFICIARIO,MUNICIPIO_BENEFICIARIO
120461,2020,4450,CENTRO UNIVERSITÁRIO METROPOLITANO DA AMAZÔNIA,BELEM,UNIFAMAZ - CENTRO UNIVERSITÁRIO METROPOLITANO ...,PARCIAL,PRESENCIAL,BIOMEDICINA,MATUTINO,924.XXX.XXX-68,F,Parda,11/07/1987,N,NORTE,PA,BELEM
32189,2020,4450,CENTRO UNIVERSITÁRIO METROPOLITANO DA AMAZÔNIA,BELEM,UNIFAMAZ - CENTRO UNIVERSITÁRIO METROPOLITANO ...,PARCIAL,PRESENCIAL,BIOMEDICINA,MATUTINO,924.XXX.XXX-68,F,Parda,11/07/1987,N,NORTE,PA,BELEM
64036,2020,1364,CENTRO UNIVERSIÁRIO UNIFTC,VITORIA DA CONQUISTA,CAMPUS - VITÓRIA DA CONQUISTA - EXPOSIÇÃO,INTEGRAL,PRESENCIAL,ADMINISTRAÇÃO,NOTURNO,863.XXX.XXX-02,M,Branca,29/04/1999,N,NORDESTE,BA,VITORIA DA CONQUISTA
153865,2020,1364,CENTRO UNIVERSIÁRIO UNIFTC,VITORIA DA CONQUISTA,CAMPUS - VITÓRIA DA CONQUISTA - EXPOSIÇÃO,INTEGRAL,PRESENCIAL,ADMINISTRAÇÃO,NOTURNO,863.XXX.XXX-02,M,Branca,29/04/1999,N,NORDESTE,BA,VITORIA DA CONQUISTA
154321,2020,385,UNIVERSIDADE SALVADOR,SALVADOR,CAMPUS LAPA,INTEGRAL,PRESENCIAL,ADMINISTRAÇÃO,NOTURNO,862.XXX.XXX-43,M,Parda,08/05/1999,N,NORDESTE,BA,SALVADOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120642,2020,3864,CENTRO UNIVERSITÁRIO UNIRB - ALAGOINHAS,ALAGOINHAS,UNIRB-ALAGOINHAS,PARCIAL,PRESENCIAL,ARQUITETURA E URBANISMO,NOTURNO,013.XXX.XXX-02,F,Preta,18/10/1985,N,NORDESTE,BA,CATU
150250,2020,322,UNIVERSIDADE PAULISTA,SANTANA DE PARNAIBA,SANTANA DE PARNAÍBA I,INTEGRAL,PRESENCIAL,DIREITO,MATUTINO,012.XXX.XXX-07,F,Preta,16/09/1982,N,SUDESTE,SP,BARUERI
108279,2020,322,UNIVERSIDADE PAULISTA,SANTANA DE PARNAIBA,SANTANA DE PARNAÍBA I,INTEGRAL,PRESENCIAL,DIREITO,MATUTINO,012.XXX.XXX-07,F,Preta,16/09/1982,N,SUDESTE,SP,BARUERI
66269,2020,1472,CENTRO UNIVERSITÁRIO LEONARDO DA VINCI,PORTO ALEGRE,POLO DE APOIO PRESENCIAL DE PORTO ALEGRE/RS - ...,INTEGRAL,EAD,GESTÃO DE RECURSOS HUMANOS,CURSO A DISTÂNCIA,000.XXX.XXX-90,F,Branca,21/04/1984,N,SUL,RS,VIAMAO


In [134]:
list_duplicated_posprocessing = []

for i in range(len(dfs)):
    list_duplicated_posprocessing.append(dfs[i][dfs[i].duplicated(keep=False)])

In [154]:
list_duplicated_posprocessing[12].sort_values(by=['cpf'], ascending=False)

,ano,sigla_uf,id_municipio,cpf,sexo,raca_cor,data_nascimento,beneficiario_deficiente,id_ies,campus,nome_municipio_ies,curso,turno_curso,tipo_bolsa,modalidade_ensino
32189,2020,PA,1501402,924.***.***-68,1,4,1987-11-07,0,4450,UNIFAMAZ - CENTRO UNIVERSITARIO METROPOLITANO ...,BELEM,BIOMEDICINA,1,2,1
120461,2020,PA,1501402,924.***.***-68,1,4,1987-11-07,0,4450,UNIFAMAZ - CENTRO UNIVERSITARIO METROPOLITANO ...,BELEM,BIOMEDICINA,1,2,1
64036,2020,BA,2933307,863.***.***-02,2,2,1999-04-29,0,1364,CAMPUS - VITORIA DA CONQUISTA - EXPOSICAO,VITORIA DA CONQUISTA,ADMINISTRACAO,3,1,1
153865,2020,BA,2933307,863.***.***-02,2,2,1999-04-29,0,1364,CAMPUS - VITORIA DA CONQUISTA - EXPOSICAO,VITORIA DA CONQUISTA,ADMINISTRACAO,3,1,1
55352,2020,BA,2927408,862.***.***-43,2,4,1999-08-05,0,385,CAMPUS LAPA,SALVADOR,ADMINISTRACAO,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87129,2020,BA,2907509,013.***.***-02,1,5,1985-10-18,0,3864,UNIRB-ALAGOINHAS,ALAGOINHAS,ARQUITETURA E URBANISMO,3,2,1
108279,2020,SP,3505708,012.***.***-07,1,5,1982-09-16,0,322,SANTANA DE PARNAIBA I,SANTANA DE PARNAIBA,DIREITO,1,1,1
150250,2020,SP,3505708,012.***.***-07,1,5,1982-09-16,0,322,SANTANA DE PARNAIBA I,SANTANA DE PARNAIBA,DIREITO,1,1,1
66269,2020,RS,4323002,000.***.***-90,1,2,1984-04-21,0,1472,POLO DE APOIO PRESENCIAL DE PORTO ALEGRE/RS - ...,PORTO ALEGRE,GESTAO DE RECURSOS HUMANOS,5,1,2


In [185]:
for i in range(len(dfs)):
    dfs[i].drop(['curso'], axis=1, inplace=True)

In [167]:
print(len(list_duplicated_preprocessing[14]))
print(len(list_duplicated_posprocessing[14]))

15939
20494


ANALISE DUPLICATES ATE AQUI

In [190]:
dfs[13].head(3)

,ano,sigla_uf,id_municipio,cpf,sexo,raca_cor,data_nascimento,beneficiario_deficiente,id_ies,campus,nome_municipio_ies,turno_curso,tipo_bolsa,modalidade_ensino
0,2018.0,PE,2611606,***57897489**,1,4,1980-07-15,0,11.0,NaN,NaN,3,1,1
1,2018.0,PE,2610707,***76246461**,2,2,2000-06-09,0,11.0,NaN,NaN,3,1,1
2,2018.0,PE,2611606,***39273455**,1,4,1998-04-11,0,11.0,NaN,NaN,2,1,1


### Saving partitioned data into specific folders

In [66]:
for df in dfs:
    ano = int(df.iloc[0, 0])
    print("Particionando ano {}".format(ano))
    for uf in ufs:
        print("Particionando {} de {}".format(uf, ano))
        df_uf = df[df['sigla_uf'] == uf]
        df_uf.drop(['ano', 'sigla_uf'], axis=1, inplace=True)
        exec("df_uf.to_csv('output/ano={}/sigla_uf={}/microdados.csv', index=False, encoding='utf-8', na_rep='', float_format='%.0f')".format(ano, uf))


Particionando ano 2009
Particionando AC de 2009
Particionando AL de 2009
Particionando AM de 2009
Particionando AP de 2009
Particionando BA de 2009
Particionando CE de 2009
Particionando DF de 2009


/Users/lucaspb/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Particionando ES de 2009
Particionando GO de 2009
Particionando MA de 2009
Particionando MG de 2009
Particionando MS de 2009
Particionando MT de 2009
Particionando PA de 2009
Particionando PB de 2009
Particionando PE de 2009
Particionando PI de 2009
Particionando PR de 2009
Particionando RJ de 2009
Particionando RN de 2009
Particionando RO de 2009
Particionando RR de 2009
Particionando RS de 2009
Particionando SC de 2009
Particionando SE de 2009
Particionando SP de 2009
Particionando TO de 2009
Particionando ano 2008
Particionando AC de 2008
Particionando AL de 2008
Particionando AM de 2008
Particionando AP de 2008
Particionando BA de 2008
Particionando CE de 2008
Particionando DF de 2008
Particionando ES de 2008
Particionando GO de 2008
Particionando MA de 2008
Particionando MG de 2008
Particionando MS de 2008
Particionando MT de 2008
Particionando PA de 2008
Particionando PB de 2008
Particionando PE de 2008
Particionando PI de 2008
Particionando PR de 2008
Particionando RJ de 2008
Pa